In [25]:
import pandas as pd
import numpy as np
import pickle
import nltk
import string
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer

In [26]:
dataFrame = pd.read_csv("/Users/Oyewole Salami/Documents/WarPeaceModel/War_Peace_Dataset.csv", sep =",")
#df = dataFrame.sample(frac = 1)
labelEn = LabelEncoder()
df['Class'] = labelEn.fit_transform(df['Class'])
df["Galtung Encoded"] = labelEn.fit_transform(df["Galtung's Criteria"])
df.describe()

,Class,Galtung Encoded
count,3026.000000,3026.000000
mean,0.590218,3.983807
std,0.491875,1.733311
min,0.000000,0.000000
25%,0.000000,3.000000
50%,1.000000,5.000000
75%,1.000000,5.000000
max,1.000000,7.000000


In [27]:
df["Galtung's Criteria"].unique()

array(['Focus on invisible effects of violence (trauma and glory, damage to structure/ culture)',
       'Focus only on visible effect of violence (killed, wounded and material damage)',
       'Focus on conflict arena, 2 parties,  1 goal (win), war general zero-sum orientation.',
       'Peace = Victory + Ceasefire', 'Peace = Non-violence + Creativity',
       "Explore conflict formation, x parties, y goals, z issues general 'win, win, orientation'.",
       'Focus on suffering all over. On aged children, women, giving voice to the voiceless.',
       "Focus on 'our' suffering; on able-bodied elite males, being their mouth-piece."],
      dtype=object)

In [28]:
df["Galtung Encoded"].unique()

array([3, 5, 2, 7, 6, 0, 4, 1])

In [29]:
df

,Headline,Galtung's Criteria,Class,Galtung Encoded
2140,The Islamic State of Iraq and the Levant (ISIL...,Focus on invisible effects of violence (trauma...,0,3
998,The attempted missile strikes is the most seri...,Focus only on visible effect of violence (kill...,1,5
221,Russia is withdrawing its troops from Syria. B...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2
2070,How football has given these Palestinian amput...,Focus on invisible effects of violence (trauma...,0,3
941,There's a large natural gas field just off Gaz...,Focus only on visible effect of violence (kill...,1,5
...,...,...,...,...
184,British government wins support for air strike...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2
992,"""Staggering."" - UN says 21 million Yemenis in ...",Focus only on visible effect of violence (kill...,1,5
1758,"BREAKING: Truce in Syria to begin at midnight,...",Peace = Victory + Ceasefire,1,7
1687,"""We hope they [ISIL] repent to God and return ...",Focus only on visible effect of violence (kill...,1,5


## Where 0 == Peace and 1 == War

In [4]:
#Number of Peace Related News
Peace = df.loc[df['Class'] == 0 ].count()
Peace

Headline    1621
Class       1621
dtype: int64

In [5]:
#Number of War Related News
War = df.loc[df['Class'] == 1 ].count()
War

Headline    1440
Class       1440
dtype: int64

In [6]:
#Missing Values
df['Class'].isnull()

1964    False
1076    False
468     False
1822    False
1133    False
        ...  
999     False
899     False
1403    False
889     False
148     False
Name: Class, Length: 3061, dtype: bool

## Preprocessing

In [6]:
#Punctuations
def removePunc(news):
    newsWithoutPunc = "".join([charac for charac in news if charac not in string.punctuation])
    return newsWithoutPunc

In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
df['Headline'] = df['Headline'].apply(lambda news: removePunc(news))
df

,Headline,Galtung's Criteria,Class,Galtung Encoded
2160,Since Jarablus we have headed west and managed...,Focus on invisible effects of violence (trauma...,0,3
283,Yemeni government troops and allies from a Sau...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2
2677,Whats going on in Syria is a miniworld war Wha...,"Focus on suffering all over. On aged children,...",0,4
1596,An Israeli army spokesperson declined to discu...,Focus only on visible effect of violence (kill...,1,5
421,The orders are expected to include restriction...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2
...,...,...,...,...
193,While we continue to monitor the terrible situ...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2
1847,In an interview with Al Jazeera last month Mor...,"Explore conflict formation, x parties, y goals...",0,0
1303,We wish it hadnt happened but it happened I ho...,Focus only on visible effect of violence (kill...,1,5
2602,The experts from the Thomson Reuters Foundatio...,"Focus on suffering all over. On aged children,...",0,4


In [9]:
#Tokenization
def tokenize(news):
    tokens = re.split('\W+', news)
    return tokens

df['tokenizedHeadline'] = df['Headline'].apply(lambda words: tokenize(words.lower()))
df

,Headline,Galtung's Criteria,Class,Galtung Encoded,tokenizedHeadline
2160,Since Jarablus we have headed west and managed...,Focus on invisible effects of violence (trauma...,0,3,"[since, jarablus, we, have, headed, west, and,..."
283,Yemeni government troops and allies from a Sau...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2,"[yemeni, government, troops, and, allies, from..."
2677,Whats going on in Syria is a miniworld war Wha...,"Focus on suffering all over. On aged children,...",0,4,"[whats, going, on, in, syria, is, a, miniworld..."
1596,An Israeli army spokesperson declined to discu...,Focus only on visible effect of violence (kill...,1,5,"[an, israeli, army, spokesperson, declined, to..."
421,The orders are expected to include restriction...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2,"[the, orders, are, expected, to, include, rest..."
...,...,...,...,...,...
193,While we continue to monitor the terrible situ...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2,"[while, we, continue, to, monitor, the, terrib..."
1847,In an interview with Al Jazeera last month Mor...,"Explore conflict formation, x parties, y goals...",0,0,"[in, an, interview, with, al, jazeera, last, m..."
1303,We wish it hadnt happened but it happened I ho...,Focus only on visible effect of violence (kill...,1,5,"[we, wish, it, hadnt, happened, but, it, happe..."
2602,The experts from the Thomson Reuters Foundatio...,"Focus on suffering all over. On aged children,...",0,4,"[the, experts, from, the, thomson, reuters, fo..."


In [10]:
#Removing Stopwords
stopwords = nltk.corpus.stopwords.words('english')
def removeStopwords(tokenizedHeadline):
    cleanedNews = [word for word in tokenizedHeadline if word not in stopwords]
    return cleanedNews
df['refinedTokens'] = df['tokenizedHeadline'].apply(lambda news: removeStopwords(news))
df

,Headline,Galtung's Criteria,Class,Galtung Encoded,tokenizedHeadline,refinedTokens
2160,Since Jarablus we have headed west and managed...,Focus on invisible effects of violence (trauma...,0,3,"[since, jarablus, we, have, headed, west, and,...","[since, jarablus, headed, west, managed, take,..."
283,Yemeni government troops and allies from a Sau...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2,"[yemeni, government, troops, and, allies, from...","[yemeni, government, troops, allies, saudiled,..."
2677,Whats going on in Syria is a miniworld war Wha...,"Focus on suffering all over. On aged children,...",0,4,"[whats, going, on, in, syria, is, a, miniworld...","[whats, going, syria, miniworld, war, people, ..."
1596,An Israeli army spokesperson declined to discu...,Focus only on visible effect of violence (kill...,1,5,"[an, israeli, army, spokesperson, declined, to...","[israeli, army, spokesperson, declined, discus..."
421,The orders are expected to include restriction...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2,"[the, orders, are, expected, to, include, rest...","[orders, expected, include, restrictions, refu..."
...,...,...,...,...,...,...
193,While we continue to monitor the terrible situ...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2,"[while, we, continue, to, monitor, the, terrib...","[continue, monitor, terrible, situation, clear..."
1847,In an interview with Al Jazeera last month Mor...,"Explore conflict formation, x parties, y goals...",0,0,"[in, an, interview, with, al, jazeera, last, m...","[interview, al, jazeera, last, month, moroccan..."
1303,We wish it hadnt happened but it happened I ho...,Focus only on visible effect of violence (kill...,1,5,"[we, wish, it, hadnt, happened, but, it, happe...","[wish, hadnt, happened, happened, hope, someth..."
2602,The experts from the Thomson Reuters Foundatio...,"Focus on suffering all over. On aged children,...",0,4,"[the, experts, from, the, thomson, reuters, fo...","[experts, thomson, reuters, foundation, ranked..."


In [11]:
#Stemming
#def stemmer(refinedTokens):
#    ps = PorterStemmer()
#    stemmedWord = [ps.stem(word) for word in refinedTokens]
#    return stemmedWord
#df['stemmedTokens'] = df['refinedTokens'].apply(lambda word: stemmer(word))
#df

In [11]:
#Lemmatization
def lemmatizer(refinedTokens):
    wn = nltk.WordNetLemmatizer()
    lemmatizedWords =  [wn.lemmatize(word) for word in refinedTokens]
    return lemmatizedWords
df['lemmatizedWords'] = df['refinedTokens'].apply(lambda word: lemmatizer(word))
df

,Headline,Galtung's Criteria,Class,Galtung Encoded,tokenizedHeadline,refinedTokens,lemmatizedWords
2160,Since Jarablus we have headed west and managed...,Focus on invisible effects of violence (trauma...,0,3,"[since, jarablus, we, have, headed, west, and,...","[since, jarablus, headed, west, managed, take,...","[since, jarablus, headed, west, managed, take,..."
283,Yemeni government troops and allies from a Sau...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2,"[yemeni, government, troops, and, allies, from...","[yemeni, government, troops, allies, saudiled,...","[yemeni, government, troop, ally, saudiled, co..."
2677,Whats going on in Syria is a miniworld war Wha...,"Focus on suffering all over. On aged children,...",0,4,"[whats, going, on, in, syria, is, a, miniworld...","[whats, going, syria, miniworld, war, people, ...","[whats, going, syria, miniworld, war, people, ..."
1596,An Israeli army spokesperson declined to discu...,Focus only on visible effect of violence (kill...,1,5,"[an, israeli, army, spokesperson, declined, to...","[israeli, army, spokesperson, declined, discus...","[israeli, army, spokesperson, declined, discus..."
421,The orders are expected to include restriction...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2,"[the, orders, are, expected, to, include, rest...","[orders, expected, include, restrictions, refu...","[order, expected, include, restriction, refuge..."
...,...,...,...,...,...,...,...
193,While we continue to monitor the terrible situ...,"Focus on conflict arena, 2 parties, 1 goal (w...",1,2,"[while, we, continue, to, monitor, the, terrib...","[continue, monitor, terrible, situation, clear...","[continue, monitor, terrible, situation, clear..."
1847,In an interview with Al Jazeera last month Mor...,"Explore conflict formation, x parties, y goals...",0,0,"[in, an, interview, with, al, jazeera, last, m...","[interview, al, jazeera, last, month, moroccan...","[interview, al, jazeera, last, month, moroccan..."
1303,We wish it hadnt happened but it happened I ho...,Focus only on visible effect of violence (kill...,1,5,"[we, wish, it, hadnt, happened, but, it, happe...","[wish, hadnt, happened, happened, hope, someth...","[wish, hadnt, happened, happened, hope, someth..."
2602,The experts from the Thomson Reuters Foundatio...,"Focus on suffering all over. On aged children,...",0,4,"[the, experts, from, the, thomson, reuters, fo...","[experts, thomson, reuters, foundation, ranked...","[expert, thomson, reuters, foundation, ranked,..."


In [12]:
#Vectorization
cv = CountVectorizer(analyzer=lemmatizer)
vectors = cv.fit_transform(df['lemmatizedWords']).toarray
print(vectors)

<bound method _cs_matrix.toarray of <3026x7437 sparse matrix of type '<class 'numpy.int64'>'
	with 45127 stored elements in Compressed Sparse Row format>>
